## Overview

Not all layers in a neural network are equally important. Some are fragile — compressing them even slightly degrades performance — while others are robust and can be heavily compressed with minimal impact. **Sensitivity analysis** measures this per-layer fragility, enabling smarter compression strategies.

The `SensitivityAnalyzer` works by compressing one layer at a time, measuring the impact on a user-provided evaluation metric, and ranking layers by their sensitivity (delta from baseline).

**Key Features:**

- Supports three compression types: **sparsity** (weight zeroing), **pruning** (structural filter removal), and **quantization** (precision reduction)
- Generates **non-uniform per-layer targets** via `to_layer_targets()` — fragile layers get less compression, robust layers get more
- Uses fasterai's `Sparsifier` and `Pruner` internally for consistent results
- Visualization with `plot()` and export to pandas with `to_dataframe()`

### When to Use

| Compression Type | `level` parameter | What it tests | Best for |
|------------------|-------------------|---------------|----------|
| **`"sparsity"`** | % of weights zeroed (e.g., 50) | Unstructured weight removal | Generating per-layer sparsity targets |
| **`"pruning"`** | % of filters removed (e.g., 30) | Structural filter pruning | Identifying layers to protect via `ignored_layers` |
| **`"quantization"`** | Bit width (e.g., 8) | Precision reduction | Finding layers that need higher precision |

In [ ]:
#| default_exp analyze.sensitivity

In [ ]:
#| export
from __future__ import annotations
import torch
import torch.nn as nn
import numpy as np
from copy import deepcopy
from dataclasses import dataclass, field, asdict
from typing import Callable, Any, Literal
from collections import OrderedDict
from fastcore.basics import store_attr

# fasterai imports (relative within fasterai package)
from fasterai.sparse.all import Sparsifier
from fasterai.prune.all import Pruner
from fasterai.core.all import large_final, Criteria, Granularities

In [ ]:
#| hide
from nbdev.showdoc import *

## Data Classes

Sensitivity results are returned as structured dataclasses for easy inspection, sorting, and export.

In [ ]:
#| export
@dataclass(slots=True)
class LayerSensitivity:
    """Sensitivity result for a single layer."""
    name: str                    # layer name
    layer_type: str              # e.g., "Conv2d", "Linear"
    params: int                  # number of parameters
    baseline_metric: float       # metric before compression
    compressed_metric: float     # metric after compression
    delta: float                 # metric change (positive = degradation)
    
    def as_dict(self) -> dict[str, Any]:
        """Convert to dictionary."""
        return asdict(self)


@dataclass(slots=True)
class SensitivityResult:
    """Structured result from sensitivity analysis."""
    compression_type: str                   # "sparsity", "pruning", "quantization"
    compression_level: float                # e.g., 50 for 50% sparsity
    baseline_metric: float                  # overall baseline metric
    layers: list[LayerSensitivity]          # per-layer results
    metric_name: str = "accuracy"           # name of the metric
    higher_is_better: bool = True           # whether higher metric is better
    _results: list[LayerSensitivity] = field(default=None, init=False, repr=False)  # for top() compatibility
    
    def __post_init__(self):
        self._results = self.layers  # for compatibility with top() pattern
    
    def as_dict(self) -> dict[str, Any]:
        """Convert to flat dictionary."""
        return {
            "compression_type": self.compression_type,
            "compression_level": self.compression_level,
            "baseline_metric": self.baseline_metric,
            "metric_name": self.metric_name,
            "higher_is_better": self.higher_is_better,
            "layers": [l.as_dict() for l in self.layers],
        }
    
    def top(
        self,
        n: int = 5,                    # number of layers to return
        *,
        most_sensitive: bool = True,   # True=highest delta (fragile), False=lowest (robust)
    ) -> list[LayerSensitivity]:
        """Return top N most or least sensitive layers."""
        sorted_layers = sorted(self.layers, key=lambda x: x.delta, reverse=most_sensitive)
        return sorted_layers[:n]
    
    def summary(
        self,
        *,
        top: int = 5,  # number of layers to show per category
    ) -> None:
        """Print a formatted summary of sensitivity analysis."""
        print(f"{'═' * 60}")
        print(f"Sensitivity Analysis: {self.compression_type} @ {self.compression_level}%")
        print(f"{'═' * 60}")
        print(f"  Baseline {self.metric_name}: {self.baseline_metric:.4f}")
        print(f"  Layers analyzed: {len(self.layers)}")
        print()
        
        # Most sensitive (fragile) layers
        print(f"  🔴 Most Sensitive (fragile):")
        for i, layer in enumerate(self.top(top, most_sensitive=True), 1):
            sign = "+" if layer.delta > 0 else ""
            print(f"     {i}. {layer.name:30} Δ={sign}{layer.delta:.4f}")
        print()
        
        # Most robust layers
        print(f"  🟢 Most Robust (compressible):")
        for i, layer in enumerate(self.top(top, most_sensitive=False), 1):
            sign = "+" if layer.delta > 0 else ""
            print(f"     {i}. {layer.name:30} Δ={sign}{layer.delta:.4f}")
    
    def to_dataframe(self):
        """Convert to pandas DataFrame."""
        import pandas as pd
        rows = [layer.as_dict() for layer in self.layers]
        return pd.DataFrame(rows)
    
    def to_layer_targets(
        self,
        model: nn.Module,          # model (used for parameter counts)
        target_pct: float = 50,    # target mean compression percentage
        min_pct: float = 0,        # minimum compression for any layer
        max_pct: float = 90,       # maximum compression for any layer
        gamma: float = 1.0,        # exponent for sensitivity scaling (higher = more differentiation)
    ) -> dict[str, float]:
        """Convert sensitivity to non-uniform per-layer compression targets.
        
        High sensitivity layers get lower compression, robust layers get higher.
        Uses parameter-weighted optimization to hit target_pct exactly.
        """
        if not self.layers:
            return {}
        
        # Convert to fractions
        target = target_pct / 100.0
        smin = min_pct / 100.0
        smax = max_pct / 100.0
        
        # Get sensitivity scores
        names = [l.name for l in self.layers]
        deltas = np.array([max(0.0, l.delta) for l in self.layers], dtype=float)
        weights = np.array([float(l.params) for l in self.layers], dtype=float)
        
        if weights.sum() == 0:
            return {n: target_pct for n in names}
        
        # Normalize sensitivity and invert (high sensitivity -> low compression)
        if np.allclose(deltas, deltas[0]):
            s0 = np.full_like(deltas, target)
        else:
            norm = (deltas - deltas.min()) / (np.ptp(deltas) + 1e-12)
            inv = (1.0 - norm) ** gamma
            s0 = smin + (smax - smin) * inv
        
        # Binary search for lambda to hit target weighted mean
        W = weights.sum()
        tgt = target * W
        
        def f(lam):
            s = np.clip(s0 + lam, smin, smax)
            return float(np.dot(weights, s))
        
        # Find lambda via bisection
        lam_lo, lam_hi = -1.0, 1.0
        while f(lam_lo) > tgt:
            lam_lo *= 2
        while f(lam_hi) < tgt:
            lam_hi *= 2
        
        for _ in range(60):
            lam_mid = 0.5 * (lam_lo + lam_hi)
            if f(lam_mid) < tgt:
                lam_lo = lam_mid
            else:
                lam_hi = lam_mid
        
        final_s = np.clip(s0 + 0.5 * (lam_lo + lam_hi), smin, smax)
        
        return {name: round(s * 100, 2) for name, s in zip(names, final_s)}
    
    def plot(
        self,
        figsize: tuple = (12, 5),  # figure size (width, height)
    ) -> None:
        """Plot sensitivity as a bar chart."""
        import matplotlib.pyplot as plt
        
        names = [l.name for l in self.layers]
        deltas = np.array([l.delta for l in self.layers], dtype=float)
        
        # Color by sensitivity
        norm = (deltas - deltas.min()) / (np.ptp(deltas) + 1e-9)
        colors = plt.cm.RdYlGn_r(norm)  # Red=sensitive, Green=robust
        
        plt.figure(figsize=figsize)
        plt.bar(range(len(deltas)), deltas, color=colors)
        plt.axhline(0, color='gray', linewidth=1.2, linestyle='--')
        plt.xticks(range(len(names)), names, rotation=60, ha='right')
        plt.ylabel(f"{self.metric_name} drop (Δ)")
        plt.title(f"Layer Sensitivity to {self.compression_type} @ {self.compression_level}%", 
                  pad=12, weight='bold')
        plt.grid(axis='y', linestyle=':', alpha=0.6)
        plt.tight_layout()
        plt.show()

In [ ]:
show_doc(LayerSensitivity)

`LayerSensitivity` holds the result for a single layer: the metric before and after compression, the delta (positive = degradation), and layer metadata. Use `as_dict()` to serialize.

---

In [ ]:
show_doc(SensitivityResult)

`SensitivityResult` aggregates all per-layer results. It provides methods to inspect, rank, visualize, and convert sensitivity data into actionable compression targets.

### Key Methods

In [ ]:
show_doc(SensitivityResult.top)

In [ ]:
show_doc(SensitivityResult.to_layer_targets)

The `to_layer_targets()` method converts sensitivity scores into a `dict[str, float]` mapping layer names to compression percentages. This dict can be passed directly to `Sparsifier.sparsify_model()` or `SparsifyCallback(sparsity=...)` for non-uniform compression.

The `gamma` parameter controls differentiation: higher values protect fragile layers more aggressively.

```python
targets = result.to_layer_targets(model, target_pct=50, min_pct=10, max_pct=80, gamma=1.5)
# {'conv1': 62.5, 'layer1.0.conv1': 65.3, 'layer1.1.conv2': 10.0, ...}
```

---

In [ ]:
show_doc(SensitivityResult.summary)
show_doc(SensitivityResult.plot)
show_doc(SensitivityResult.to_dataframe)

---

## SensitivityAnalyzer

The `SensitivityAnalyzer` class provides full control over the analysis process. For quick one-off analysis, see `analyze_sensitivity()` below.

In [ ]:
#| export
class SensitivityAnalyzer:
    """Analyze per-layer sensitivity to compression methods.
    
    Uses fasterai's Sparsifier for sparsity analysis and Pruner for structural pruning.
    Supports sparsity (weight zeroing), pruning (structural), and quantization.
    """
    
    VALID_COMPRESSIONS = frozenset({"sparsity", "pruning", "quantization"})
    COMPRESSIBLE_LAYERS = Granularities.available_modules()  # Use fasterai's layer registry
    
    def __init__(
        self,
        model: nn.Module,                              # model to analyze
        sample: torch.Tensor,                          # example input (for Pruner dependency analysis)
        eval_fn: Callable[[nn.Module], float],         # evaluation function returning metric
        *,
        criteria: Criteria = large_final,              # fasterai criteria for importance scoring
        higher_is_better: bool = True,                 # whether higher metric values are better
        metric_name: str = "accuracy",                 # name of the metric for display
        device: str | torch.device | None = None,      # device for computation
        calibration_data: torch.Tensor | None = None,  # for observer-based quantization
    ):
        store_attr()
        self.device = device or next(model.parameters()).device
        self._results: SensitivityResult | None = None
        self._sparsifier: Sparsifier | None = None
        self._activation_hooks: list[Any] = []
        self._activation_quantize_config: dict[str, bool] = {}
    
    def _get_compressible_layers(self) -> list[tuple[str, nn.Module]]:
        """Get all compressible layers (Conv2d, Linear, etc.)."""
        return [
            (name, module) 
            for name, module in self.model.named_modules()
            if isinstance(module, self.COMPRESSIBLE_LAYERS)
            and hasattr(module, 'weight') and module.weight is not None
        ]
    
    def _init_sparsifier(
        self,
        granularity: str = "weight",  # sparsity granularity
    ) -> None:
        """Initialize fasterai Sparsifier (saves initial weights for all layers)."""
        if self._sparsifier is None:
            self._sparsifier = Sparsifier(
                self.model,
                granularity=granularity,
                context='local',
                criteria=self.criteria,
            )
    
    def _cleanup_sparsifier(self) -> None:
        """Remove sparsifier buffers from model."""
        if self._sparsifier is not None:
            self._sparsifier._clean_buffers()
            self._sparsifier = None
    
    def _apply_sparsity(
        self,
        module: nn.Module,  # layer to sparsify
        level: float,       # sparsity percentage (0-100)
    ) -> None:
        """Apply sparsity using fasterai Sparsifier."""
        self._sparsifier.sparsify_layer(module, level)
    
    def _restore_layer(
        self,
        module: nn.Module,  # layer to restore
    ) -> None:
        """Restore a single layer from saved initial weights."""
        if hasattr(module, '_init_weights'):
            module.weight.data.copy_(module._init_weights)
        if hasattr(module, '_init_biases') and module._init_biases is not None:
            module.bias.data.copy_(module._init_biases)
        if hasattr(module, '_mask'):
            del module._buffers['_mask']
    
    def _clone_model(self) -> nn.Module:
        """Create a clean copy of the model (avoids deepcopy issues with non-leaf tensors)."""
        import io
        buffer = io.BytesIO()
        torch.save(self.model, buffer)
        buffer.seek(0)
        model_copy = torch.load(buffer, weights_only=False)
        model_copy.eval()
        return model_copy
    
    def _apply_structural_pruning(
        self, 
        target_name: str,  # name of layer to prune
        level: float,      # pruning ratio (0-100)
    ) -> nn.Module:
        """Apply structural pruning to a single layer using fasterai Pruner.
        
        Returns a copy of the model with only the target layer pruned.
        Uses state_dict cloning to avoid deepcopy issues with non-leaf tensors.
        """
        try:
            model_copy = self._clone_model()
        except Exception:
            # Fallback to deepcopy if clone fails
            model_copy = deepcopy(self.model)
        
        all_layers = []
        target_module = None
        for name, module in model_copy.named_modules():
            if isinstance(module, self.COMPRESSIBLE_LAYERS):
                all_layers.append(module)
                if name == target_name:
                    target_module = module
        
        ignored_layers = [m for m in all_layers if m is not target_module]
        
        try:
            pruner = Pruner(
                model_copy,
                pruning_ratio=level,
                context='local',
                criteria=self.criteria,
                ignored_layers=ignored_layers,
                example_inputs=self.sample,
            )
            pruner.prune_model()
        except Exception as e:
            import warnings
            warnings.warn(f"Structural pruning failed for {target_name}: {e}")
            return model_copy
        
        return model_copy
    
    # ─── Quantization helpers ────────────────────────────────────────────────────
    
    def _compute_qparams_symmetric(
        self,
        tensor: torch.Tensor,       # tensor to compute qparams for
        bits: int = 8,              # quantization bits
        per_channel: bool = False,  # per-channel or per-tensor
        channel_axis: int = 0,      # axis for per-channel quantization
    ) -> tuple[torch.Tensor, torch.Tensor]:
        """Compute scale and zero_point for symmetric quantization."""
        qmin, qmax = -(2 ** (bits - 1)), 2 ** (bits - 1) - 1
        
        if per_channel and tensor.dim() > 1:
            dims = list(range(tensor.dim()))
            dims.remove(channel_axis)
            amax = tensor.abs()
            for dim in sorted(dims, reverse=True):
                amax = amax.max(dim=dim).values
            scale = amax / qmax
            scale = torch.clamp(scale, min=1e-8)
            zero_point = torch.zeros_like(scale, dtype=torch.int32)
        else:
            amax = tensor.abs().max()
            scale = torch.tensor([max(amax.item() / qmax, 1e-8)], device=tensor.device)
            zero_point = torch.tensor([0], dtype=torch.int32, device=tensor.device)
        
        return scale, zero_point
    
    def _compute_qparams_asymmetric(
        self,
        tensor: torch.Tensor,       # tensor to compute qparams for
        bits: int = 8,              # quantization bits
        per_channel: bool = False,  # per-channel or per-tensor
        channel_axis: int = 0,      # axis for per-channel quantization
    ) -> tuple[torch.Tensor, torch.Tensor]:
        """Compute scale and zero_point for asymmetric quantization."""
        qmin, qmax = 0, 2 ** bits - 1
        
        if per_channel and tensor.dim() > 1:
            dims = list(range(tensor.dim()))
            dims.remove(channel_axis)
            t_min, t_max = tensor.clone(), tensor.clone()
            for dim in sorted(dims, reverse=True):
                t_min = t_min.min(dim=dim).values
                t_max = t_max.max(dim=dim).values
        else:
            t_min, t_max = tensor.min(), tensor.max()
        
        scale = (t_max - t_min) / (qmax - qmin)
        scale = torch.clamp(scale, min=1e-8)
        zero_point = torch.clamp(torch.round(-t_min / scale), qmin, qmax).to(torch.int32)
        
        if not per_channel or tensor.dim() <= 1:
            scale = scale.view(1) if scale.dim() == 0 else scale
            zero_point = zero_point.view(1) if zero_point.dim() == 0 else zero_point
        
        return scale, zero_point
    
    def _fake_quantize_per_channel(
        self,
        tensor: torch.Tensor,      # tensor to quantize
        bits: int = 8,             # quantization bits
        symmetric: bool = True,    # symmetric or asymmetric
        channel_axis: int = 0,     # axis for per-channel quantization
    ) -> torch.Tensor:
        """Apply fake quantization (per-channel)."""
        qmin = -(2 ** (bits - 1)) if symmetric else 0
        qmax = (2 ** (bits - 1)) - 1 if symmetric else (2 ** bits) - 1
        
        if symmetric:
            scale, zero_point = self._compute_qparams_symmetric(
                tensor, bits, per_channel=True, channel_axis=channel_axis
            )
        else:
            scale, zero_point = self._compute_qparams_asymmetric(
                tensor, bits, per_channel=True, channel_axis=channel_axis
            )
        
        return torch.fake_quantize_per_channel_affine(
            tensor, scale, zero_point, channel_axis, qmin, qmax
        )
    
    def _fake_quantize_per_tensor(
        self,
        tensor: torch.Tensor,    # tensor to quantize
        bits: int = 8,           # quantization bits
        symmetric: bool = True,  # symmetric or asymmetric
    ) -> torch.Tensor:
        """Apply fake quantization (per-tensor)."""
        qmin = -(2 ** (bits - 1)) if symmetric else 0
        qmax = (2 ** (bits - 1)) - 1 if symmetric else (2 ** bits) - 1
        
        if symmetric:
            scale, zero_point = self._compute_qparams_symmetric(tensor, bits, per_channel=False)
        else:
            scale, zero_point = self._compute_qparams_asymmetric(tensor, bits, per_channel=False)
        
        return torch.fake_quantize_per_tensor_affine(
            tensor, scale.item(), int(zero_point.item()), qmin, qmax
        )
    
    def _apply_weight_quantization(
        self, 
        module: nn.Module,       # layer to quantize
        bits: int = 8,           # quantization bits
        per_channel: bool = True,  # per-channel or per-tensor
    ) -> None:
        """Apply weight quantization using fake_quantize."""
        weight = module.weight.data
        if per_channel and weight.dim() > 1:
            quantized = self._fake_quantize_per_channel(weight, bits, symmetric=True, channel_axis=0)
        else:
            quantized = self._fake_quantize_per_tensor(weight, bits, symmetric=True)
        weight.copy_(quantized)
    
    def _create_activation_quantize_hook(
        self,
        layer_name: str,  # layer name for config lookup
        bits: int = 8,    # quantization bits
    ):
        """Create a forward hook that quantizes activations."""
        def hook(module, input, output):
            if self._activation_quantize_config.get(layer_name, False):
                return self._fake_quantize_per_tensor(output, bits, symmetric=False)
            return output
        return hook
    
    def _setup_activation_hooks(
        self,
        bits: int = 8,  # quantization bits
    ) -> None:
        """Register activation quantization hooks on all layers."""
        self._remove_activation_hooks()
        for name, module in self._get_compressible_layers():
            hook = self._create_activation_quantize_hook(name, bits)
            handle = module.register_forward_hook(hook)
            self._activation_hooks.append(handle)
            self._activation_quantize_config[name] = False
    
    def _remove_activation_hooks(self) -> None:
        """Remove all activation quantization hooks."""
        for handle in self._activation_hooks:
            handle.remove()
        self._activation_hooks = []
        self._activation_quantize_config = {}
    
    # ─── Main analysis method ────────────────────────────────────────────────────
    
    def analyze(
        self,
        compression: Literal["sparsity", "pruning", "quantization"] = "sparsity",  # compression type
        level: float = 50,                    # compression level (% for sparsity/pruning, bits for quant)
        *,
        granularity: str = "weight",          # granularity for sparsity (fasterai granularities)
        layers: list[str] | None = None,      # specific layer names to analyze (None = all)
        quant_per_channel: bool = True,       # use per-channel quantization
        quant_activations: bool = False,      # also quantize activations
        verbose: bool = True,                 # print progress
    ) -> SensitivityResult:
        """Analyze per-layer sensitivity to compression."""
        if compression not in self.VALID_COMPRESSIONS:
            raise ValueError(f"compression must be one of {self.VALID_COMPRESSIONS}")
        
        self.model.eval()
        
        if compression == "sparsity":
            self._init_sparsifier(granularity)
        
        if compression == "quantization" and quant_activations:
            bits = int(level) if level > 1 else 8
            self._setup_activation_hooks(bits)
        
        if verbose:
            print(f"Computing baseline {self.metric_name}...", end=" ", flush=True)
        baseline = self.eval_fn(self.model)
        if verbose:
            print(f"{baseline:.4f}")
        
        all_layers = self._get_compressible_layers()
        if layers is not None:
            all_layers = [(n, m) for n, m in all_layers if n in layers]
        
        mode_info = ""
        if compression == "quantization":
            mode_info = f" (per-{'channel' if quant_per_channel else 'tensor'}"
            mode_info += f", {'weights+activations' if quant_activations else 'weights only'})"
        elif compression == "sparsity":
            mode_info = f" (granularity={granularity}, criteria={self.criteria.f.__name__})"
        elif compression == "pruning":
            mode_info = f" (structural, criteria={self.criteria.f.__name__})"
        
        if verbose:
            unit = 'bits' if compression == 'quantization' else '%'
            print(f"Analyzing {len(all_layers)} layers for {compression} @ {level}{unit}{mode_info}")
        
        results: list[LayerSensitivity] = []
        
        for i, (name, module) in enumerate(all_layers):
            if verbose:
                print(f"  [{i+1}/{len(all_layers)}] {name}...", end=" ", flush=True)
            
            if compression == "sparsity":
                self._apply_sparsity(module, level)
                compressed_metric = self.eval_fn(self.model)
                self._restore_layer(module)
                param_count = module.weight.numel()
                
            elif compression == "pruning":
                pruned_model = self._apply_structural_pruning(name, level)
                compressed_metric = self.eval_fn(pruned_model)
                param_count = module.weight.numel()
                del pruned_model
                
            elif compression == "quantization":
                saved_weight = module.weight.data.clone()
                saved_bias = module.bias.data.clone() if module.bias is not None else None
                
                bits = int(level) if level > 1 else 8
                self._apply_weight_quantization(module, bits, per_channel=quant_per_channel)
                
                if quant_activations:
                    self._activation_quantize_config[name] = True
                
                compressed_metric = self.eval_fn(self.model)
                
                if quant_activations:
                    self._activation_quantize_config[name] = False
                
                module.weight.data.copy_(saved_weight)
                if saved_bias is not None:
                    module.bias.data.copy_(saved_bias)
                param_count = module.weight.numel()
            
            if self.higher_is_better:
                delta = baseline - compressed_metric
            else:
                delta = compressed_metric - baseline
            
            if verbose:
                sign = "+" if delta > 0 else ""
                print(f"Δ={sign}{delta:.4f}")
            
            results.append(LayerSensitivity(
                name=name,
                layer_type=module.__class__.__name__,
                params=param_count,
                baseline_metric=baseline,
                compressed_metric=compressed_metric,
                delta=delta,
            ))
        
        if compression == "sparsity":
            self._cleanup_sparsifier()
        if compression == "quantization" and quant_activations:
            self._remove_activation_hooks()
        
        compression_desc = compression
        if compression == "quantization":
            compression_desc = f"quantization-{int(level) if level > 1 else 8}bit"
            if quant_activations:
                compression_desc += "+act"
        
        self._results = SensitivityResult(
            compression_type=compression_desc,
            compression_level=level,
            baseline_metric=baseline,
            layers=results,
            metric_name=self.metric_name,
            higher_is_better=self.higher_is_better,
        )
        
        if verbose:
            print(f"✓ Analysis complete")
        
        return self._results
    
    def sweep(
        self,
        compression: Literal["sparsity", "pruning", "quantization"] = "sparsity",  # compression type
        levels: list[float] | None = None,  # compression levels to test (default: [25, 50, 75])
        **kwargs,
    ) -> list[SensitivityResult]:
        """Run sensitivity analysis at multiple compression levels."""
        if levels is None:
            levels = [25, 50, 75]
        results = []
        for level in levels:
            print(f"\n{'='*60}")
            unit = 'bits' if compression == 'quantization' else '%'
            print(f"Sweep: {compression} @ {level}{unit}")
            print(f"{'='*60}")
            result = self.analyze(compression, level, **kwargs)
            results.append(result)
        return results

In [ ]:
show_doc(SensitivityAnalyzer)

The `eval_fn` should take a `nn.Module` and return a scalar metric (e.g., accuracy, loss). The analyzer will call it once per layer, so it should be reasonably fast — a forward pass on a small validation batch is typical.

### Key Methods

In [ ]:
show_doc(SensitivityAnalyzer.analyze)

For **quantization** analysis, additional parameters control the behavior:

- `quant_per_channel=True` — per-channel quantization (more accurate, standard for weights)
- `quant_activations=False` — set to `True` to also quantize activations (slower but more realistic)
- `level` is interpreted as **bit width** (e.g., 8 for INT8) instead of percentage

---

In [ ]:
show_doc(SensitivityAnalyzer.sweep)

---

## Convenience Function

For quick one-off analysis without creating an analyzer instance:

In [ ]:
#| export
def analyze_sensitivity(
    model: nn.Module,                    # model to analyze
    sample: torch.Tensor,                # example input tensor
    eval_fn: Callable[[nn.Module], float],  # evaluation function returning metric
    compression: Literal["sparsity", "pruning", "quantization"] = "sparsity",  # compression type
    level: float = 50,                   # compression level (% for sparsity/pruning, bits for quant)
    *,
    criteria: Criteria = large_final,    # fasterai criteria for importance scoring
    higher_is_better: bool = True,       # whether higher metric values are better
    metric_name: str = "accuracy",       # name of the metric for display
    granularity: str = "weight",         # granularity for sparsity
    verbose: bool = True,                # print progress
    **kwargs,
) -> SensitivityResult:
    """One-line sensitivity analysis using fasterai compression methods."""
    analyzer = SensitivityAnalyzer(
        model, sample, eval_fn, 
        criteria=criteria, 
        higher_is_better=higher_is_better,
        metric_name=metric_name,
    )
    return analyzer.analyze(compression, level, granularity=granularity, verbose=verbose, **kwargs)

In [ ]:
show_doc(analyze_sensitivity)

### Usage Example

```python
from fasterai.analyze.sensitivity import analyze_sensitivity

result = analyze_sensitivity(model, sample, eval_fn, compression="sparsity", level=50)

# Inspect results
result.summary()                          # formatted console output
fragile = result.top(5, most_sensitive=True)  # most sensitive layers

# Generate per-layer targets for non-uniform compression
targets = result.to_layer_targets(model, target_pct=50, min_pct=10, max_pct=80)
# Pass directly to Sparsifier: sparsifier.sparsify_model(sparsity=targets)
```

---

## See Also

- [Sensitivity Tutorial](tutorials/analyze/sensitivity.html) - Step-by-step guide with real examples on ResNet18
- [Sparsifier](../sparse/sparsifier.html) - Apply non-uniform sparsity using `to_layer_targets()` output
- [Pruner](../prune/pruner.html) - Structural pruning (use `top()` to find layers to protect)
- [Criteria](../core/criteria.html) - Importance scoring methods used during analysis
- [Schedules](../core/schedules.html) - Control compression progression during training

In [ ]:
#| hide
from fastcore.test import *
import torch.nn as nn

def _test_model():
    return nn.Sequential(
        nn.Conv2d(3, 16, 3, padding=1),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.Conv2d(16, 32, 3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.AdaptiveAvgPool2d(1),
        nn.Flatten(),
        nn.Linear(32, 10)
    )

model = _test_model()
sample = torch.randn(2, 3, 8, 8)
def _eval(m):
    m.eval()
    with torch.no_grad(): return m(sample).abs().mean().item()

analyzer = SensitivityAnalyzer(model, sample, _eval)

# Invalid compression raises ValueError
with ExceptionExpected(ValueError):
    analyzer.analyze('invalid', 50, verbose=False)

# Result structure for sparsity analysis
result = analyzer.analyze('sparsity', 30, verbose=False)
assert isinstance(result, SensitivityResult)
test_eq(result.compression_level, 30)
assert len(result.layers) > 0
assert isinstance(result.layers[0], LayerSensitivity)

# top() sorted correctly (most sensitive first)
top3 = result.top(3, most_sensitive=True)
for i in range(len(top3)-1):
    assert top3[i].delta >= top3[i+1].delta

# to_layer_targets returns dict with layer names
sched = result.to_layer_targets(model, target_pct=50)
assert isinstance(sched, dict)
assert len(sched) == len(result.layers)
# All values should be percentages in valid range
for v in sched.values():
    assert 0 <= v <= 90

# LayerSensitivity.as_dict returns proper dict
d = result.layers[0].as_dict()
assert 'name' in d
assert 'delta' in d
assert 'params' in d

# SensitivityResult.as_dict works
full_d = result.as_dict()
assert 'compression_type' in full_d
assert 'layers' in full_d

# analyze_sensitivity convenience function works
result2 = analyze_sensitivity(
    _test_model(), sample, _eval,
    compression='sparsity', level=20, verbose=False
)
assert isinstance(result2, SensitivityResult)

In [ ]:
#| hide
#| slow
# Full sensitivity analysis on a larger model
from torchvision.models import resnet18

_model_lg = resnet18(weights=None)
_sample_lg = torch.randn(2, 3, 32, 32)
def _eval_lg(m):
    m.eval()
    with torch.no_grad(): return m(_sample_lg).abs().mean().item()

_result_lg = analyze_sensitivity(_model_lg, _sample_lg, _eval_lg,
                                 compression='sparsity', level=30, verbose=False)
assert isinstance(_result_lg, SensitivityResult)
assert len(_result_lg.layers) > 3  # resnet18 has many conv layers

_sched_lg = _result_lg.to_layer_targets(_model_lg, target_pct=30)
assert len(_sched_lg) > 3
for v in _sched_lg.values():
    assert 0 <= v <= 90

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()